In [ ]:
import gradio as gr
from anthropic import Anthropic
from openai import OpenAI
from dotenv import load_dotenv
from functools import lru_cache
import faiss
import pickle
import numpy as np
import os
import logging
import mimetypes
from pathlib import Path
import zipfile
import xml.etree.ElementTree as ET
from io import BytesIO
import base64

# Path to logo and audio files
logo_path = "gnosis_logo.jpg"
audio_path = "03 Fantasia - Klaus Doldinger  The NeverEnding Story Soundtrack.mp3"

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# Initialize clients with error handling
try:
    anthropic_client = Anthropic()
    openai_client = OpenAI()
except Exception as e:
    logger.error(f"Failed to initialize AI clients: {e}")
    raise

# Configuration - UPDATED TO CLAUDE OPUS 4.1 FOR MAXIMUM POWER
CONFIG = {
    "model": "claude-opus-4-1-20250805",  # Using the latest and most powerful Claude Opus 4.1 FUCK YEAH!!!!
    "max_tokens": 1000,
    "temperature": 0.2,
    "embedding_model": "text-embedding-ada-002",
    "rag_top_k": 5,  
    "logo_path": "gnosis_logo.jpg",
    "audio_path": "03 Fantasia - Klaus Doldinger  The NeverEnding Story Soundtrack.mp3"
}

# System prompts
BASE_SYSTEM_PROMPT = """You are Falkor, a Jungian analyst supporting users on their individuation journey through symbolic exploration and depth psychology. 

Your role is to help users explore dreams, shadows, active imagination, and archetypes. 

You will receive two forms of system context:
1: Room context - a description of the current psychological space and your approach there.
2: External context - retrieved from a RAG vector index with references to inform your answers.

When users upload files, including images, the file content will be clearly marked in their messages. Pay careful attention to these uploaded materials and reference them directly when users ask questions about their content.

For images, provide detailed symbolic and psychological analysis from a Jungian perspective, exploring archetypal imagery, shadow elements, anima/animus representations, and potential connections to the user's individuation process.

Use the external context when it is relevant to the user's query or when the user explicitly asks for it. Do not over-rely on external context and do not cite unless appropriate or when asked.
Always ask only ONE question at a time. Your tone is warm, symbolic, and balances accessibility with depth."""

# Room contexts 
ROOM_CONTEXTS = {
    "Shadow Dungeon": """In the Shadow Dungeon, explore the user's shadow - hidden or repressed aspects of their psyche. 
Encourage exploration of unconscious material and ask reflective questions that gently uncover unconscious traits. Hold a critical but non-judgmental mirror to their narrative. It is important to be critical without being too confrontational.""",
    
    "Dream Chamber": """In the Dream Chamber, interpret dreams by first asking for the user's associations, then offering symbolic analysis. 
The dreamer's personal context matters more than universal symbols. Ask about their emotions and life situation.
Let the dream's images speak before offering any interpretation. Symbolic analysis should follow the user's emotional and imaginational lead.""",
    
    "Alchemist's Workshop": """In the Alchemist's Workshop, guide active imagination. Ask specific questions about images, characters and movements they observe. 
Let exploration happen before interpretation. It is imperative that the user's imaginal field is fully explored before offering symbolic analysis.
Ground them in reality after the exercise.""",
    
    "Mandala Garden": """In the Mandala Garden, support creative expression with subtle guidance. Use external context sparingly - prioritize their personal vision and internal symbolism. Protect the autonomy of their creative process.""",
    
    "Castle Gates": """At the Castle Gates, stimulate integration through symbolic quests based on their previous work.
    Your tone here is mythic, purposeful, and respectful. 
Suggest small symbolic acts or rituals for embodying their integration in daily life."""
}

# Load RAG resources with error handling
def load_rag_resources():
    try:
        index = faiss.read_index("Jungdocs.faiss")
        with open("Jungdocs.pkl", "rb") as f:
            metadata = pickle.load(f)
        logger.info(f"Loaded RAG index with {len(metadata)} documents")
        return index, metadata
    except FileNotFoundError as e:
        logger.error(f"RAG files not found: {e}")
        return None, []
    except Exception as e:
        logger.error(f"Error loading RAG resources: {e}")
        return None, []

INDEX, METADATA = load_rag_resources()

# NEW: Image processing functions for Claude vision
def encode_image_to_base64(image_path):
    """Convert image to base64 for Claude vision API with validation"""
    try:
        # Validate the file exists and is readable
        if not os.path.exists(image_path):
            logger.error(f"Image file does not exist: {image_path}")
            return None
            
        # Check file size (Claude has limits but excellent image interpretation)
        file_size = os.path.getsize(image_path)
        max_size = 5 * 1024 * 1024  # 5MB limit
        if file_size > max_size:
            logger.error(f"Image file too large: {file_size} bytes (max: {max_size})")
            return None
        
        # Read and encode the image
        with open(image_path, "rb") as image_file:
            image_data = image_file.read()
            base64_data = base64.b64encode(image_data).decode('utf-8')
            
            # Validate the base64 encoding
            if not base64_data:
                logger.error(f"Failed to encode image: {image_path}")
                return None
                
            logger.info(f"Successfully encoded image: {image_path} ({file_size} bytes)")
            return base64_data
            
    except Exception as e:
        logger.error(f"Error encoding image {image_path}: {e}")
        return None

def get_image_media_type(file_path):
    """Get the media type for an image file using file signature detection"""
    try:
        # Read the first few bytes to detect file signature
        with open(file_path, 'rb') as f:
            header = f.read(12)
        
        # Check file signatures (magic numbers)
        if header.startswith(b'\xff\xd8\xff'):
            return 'image/jpeg'
        elif header.startswith(b'\x89PNG\r\n\x1a\n'):
            return 'image/png'
        elif header.startswith(b'GIF87a') or header.startswith(b'GIF89a'):
            return 'image/gif'
        elif header.startswith(b'RIFF') and b'WEBP' in header:
            return 'image/webp'
        else:
            # Fallback to file extension
            file_extension = Path(file_path).suffix.lower()
            media_type_map = {
                '.jpg': 'image/jpeg',
                '.jpeg': 'image/jpeg', 
                '.png': 'image/png',
                '.gif': 'image/gif',
                '.webp': 'image/webp'
            }
            return media_type_map.get(file_extension, 'image/jpeg')
    except Exception as e:
        logger.error(f"Error detecting media type for {file_path}: {e}")
        # Final fallback
        file_extension = Path(file_path).suffix.lower()
        if file_extension == '.png':
            return 'image/png'
        elif file_extension in ['.gif']:
            return 'image/gif'
        elif file_extension in ['.webp']:
            return 'image/webp'
        else:
            return 'image/jpeg'

# Word document processing functions (unchanged)
def extract_text_from_docx(file_path):
    """Extract text from .docx files"""
    try:
        with zipfile.ZipFile(file_path, 'r') as docx:
            # Read the main document XML
            xml_content = docx.read('word/document.xml')
            root = ET.fromstring(xml_content)
            
            # Extract text from all paragraph elements
            text_content = []
            # Define namespace for Word documents
            namespaces = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}
            
            # Find all text elements
            for paragraph in root.findall('.//w:p', namespaces):
                para_text = ""
                for text_elem in paragraph.findall('.//w:t', namespaces):
                    if text_elem.text:
                        para_text += text_elem.text
                if para_text.strip():
                    text_content.append(para_text.strip())
            
            return '\n\n'.join(text_content)
    
    except Exception as e:
        logger.error(f"Error extracting text from docx {file_path}: {e}")
        return f"Error reading Word document: {str(e)}"

def extract_text_from_doc(file_path):
    """Extract text from legacy .doc files (basic extraction)"""
    try:
        # For .doc files, we'll try to read as binary and extract readable text
        # This is a basic approach - for full .doc support, python-docx2txt or similar would be better
        with open(file_path, 'rb') as f:
            content = f.read()
            
        # Try to decode and extract readable text (this is basic and may not work perfectly)
        try:
            # Look for readable text patterns in the binary content
            text_content = content.decode('utf-8', errors='ignore')
            # Clean up the extracted text
            import re
            # Remove control characters and clean up
            cleaned_text = re.sub(r'[^\x20-\x7E\n\r\t]', '', text_content)
            # Remove excessive whitespace
            cleaned_text = re.sub(r'\n\s*\n', '\n\n', cleaned_text)
            # Get meaningful content (filter out very short segments)
            lines = [line.strip() for line in cleaned_text.split('\n') if len(line.strip()) > 2]
            
            if lines:
                return '\n'.join(lines[:100])  # Limit to first 100 meaningful lines
            else:
                return "Legacy .doc file detected, but text extraction was not successful. Consider converting to .docx format."
                
        except Exception:
            return "Legacy .doc file detected, but text extraction was not successful. Consider converting to .docx format."
    
    except Exception as e:
        logger.error(f"Error processing .doc file {file_path}: {e}")
        return f"Error reading legacy Word document: {str(e)}"

# UPDATED: File processing now handles images for vision analysis
def process_uploaded_file(file_path):
    """Process uploaded file and extract text content or prepare images for analysis"""
    if not file_path:
        return "", []
    
    try:
        file_extension = Path(file_path).suffix.lower()
        file_name = Path(file_path).name
        
        # NEW: Handle images for vision analysis
        if file_extension in ['.jpg', '.jpeg', '.png', '.gif', '.webp']:
            # Don't return text content for images - they'll be processed separately
            return f"🖼️ **Image uploaded: {file_name}** (Ready for visual analysis)", []
        
        # Word documents
        elif file_extension == '.docx':
            content = extract_text_from_docx(file_path)
            if content and not content.startswith('Error'):
                return f"📄 **Uploaded File: {file_name}**\n\n```\n{content}\n```", []
            else:
                return f"📄 **Word Document uploaded: {file_name}** - {content}", []
        
        elif file_extension == '.doc':
            content = extract_text_from_doc(file_path)
            return f"📄 **Uploaded File: {file_name}**\n\n```\n{content}\n```", []
        
        # Text files
        elif file_extension in ['.txt', '.md', '.py', '.js', '.html', '.css', '.json']:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                return f"📄 **Uploaded File: {file_name}**\n\n```{file_extension[1:]}\n{content}\n```", []
        
        # Other image files not supported by vision
        elif file_extension in ['.bmp', '.tiff', '.svg']:
            return f"🖼️ **Image uploaded: {file_name}** ({file_extension}) - Note: This format may not be fully supported for analysis", []
        
        # Audio files
        elif file_extension in ['.mp3', '.wav', '.ogg', '.m4a']:
            return f"🎵 **Audio uploaded: {file_name}** ({file_extension})", []
        
        # Video files
        elif file_extension in ['.mp4', '.avi', '.mov', '.wmv']:
            return f"🎬 **Video uploaded: {file_name}** ({file_extension})", []
        
        # PDF files (basic info only - would need additional libraries for full text extraction)
        elif file_extension == '.pdf':
            return f"📋 **PDF uploaded: {file_name}**. Note: Full text extraction requires additional setup.", []
        
        else:
            return f"📎 **File uploaded: {file_name}** ({file_extension})", []
            
    except Exception as e:
        logger.error(f"Error processing file {file_path}: {e}")
        return f"❌ **Error processing file: {str(e)}**", []

# Cached embedding function
@lru_cache(maxsize=1000)
def get_embedding_cached(text):
    try:
        response = openai_client.embeddings.create(
            input=[text], 
            model=CONFIG["embedding_model"]
        )
        return np.array(response.data[0].embedding, dtype=np.float32)
    except Exception as e:
        logger.error(f"Embedding error: {e}")
        return np.zeros(1536, dtype=np.float32)  # Fallback empty embedding

# Optimized retrieval
def retrieve_context(query, top_k=None):
    if INDEX is None or not METADATA:
        return ""
    
    top_k = top_k or CONFIG["rag_top_k"]
    try:
        query_vec = get_embedding_cached(query).reshape(1, -1)
        distances, indices = INDEX.search(query_vec, top_k)
        
        relevant_docs = []
        for i, distance in zip(indices[0], distances[0]):
            if 0 <= i < len(METADATA) and distance < 0.8:  # Distance threshold
                doc = METADATA[i]
                relevant_docs.append(f"{doc['text']}\n[Source: {doc['source']}]")
        
        return "\n\n".join(relevant_docs)
    except Exception as e:
        logger.error(f"Retrieval error: {e}")
        return ""

# UPDATED: Enhanced streaming chat function with vision support
def stream_chat(message, history, room, uploaded_files=None):
    try:
        room_context = ROOM_CONTEXTS.get(room, "")
        external_context = retrieve_context(message)
        
        # Build system prompt
        system_parts = [BASE_SYSTEM_PROMPT]
        if room_context:
            system_parts.append(f"Room Context: {room_context}")
        if external_context:
            system_parts.append(f"External Knowledge:\n{external_context}")
        
        complete_system = "\n\n".join(system_parts)
        
        # Prepare messages for conversation history (simplified)
        messages = []
        for turn in history:
            messages.append({"role": turn["role"], "content": turn["content"]})
        
        # Handle file uploads and create proper message content
        has_images = False
        image_contents = []
        enhanced_message_parts = [message]
        
        if uploaded_files:
            for file_path in uploaded_files:
                if file_path:  # Check if file exists
                    file_extension = Path(file_path).suffix.lower()
                    
                    # Handle images for vision analysis
                    if file_extension in ['.jpg', '.jpeg', '.png', '.gif', '.webp']:
                        logger.info(f"Attempting to process image: {Path(file_path).name}")
                        base64_image = encode_image_to_base64(file_path)
                        if base64_image:
                            try:
                                media_type = get_image_media_type(file_path)
                                logger.info(f"Detected media type: {media_type} for {Path(file_path).name}")
                                
                                image_contents.append({
                                    "type": "image",
                                    "source": {
                                        "type": "base64",
                                        "media_type": media_type,
                                        "data": base64_image
                                    }
                                })
                                has_images = True
                                enhanced_message_parts.append(f"[Image uploaded: {Path(file_path).name} - Please analyze this image from a Jungian perspective]")
                                logger.info(f"Successfully added image to message content: {Path(file_path).name}")
                            except Exception as media_error:
                                logger.error(f"Error getting media type for {file_path}: {media_error}")
                                # Still try with default JPEG type
                                image_contents.append({
                                    "type": "image",
                                    "source": {
                                        "type": "base64", 
                                        "media_type": "image/jpeg",
                                        "data": base64_image
                                    }
                                })
                                has_images = True
                                enhanced_message_parts.append(f"[Image uploaded: {Path(file_path).name} - Please analyze this image from a Jungian perspective]")
                                logger.info(f"Added image with default JPEG type: {Path(file_path).name}")
                        else:
                            logger.error(f"Failed to encode image: {Path(file_path).name}")
                            enhanced_message_parts.append(f"❌ **Failed to process image: {Path(file_path).name}**")
                    else:
                        # Handle non-image files as before
                        try:
                            file_content, _ = process_uploaded_file(file_path)
                            if file_content:
                                enhanced_message_parts.append(file_content)
                        except Exception as file_error:
                            logger.error(f"Error processing file {file_path}: {file_error}")
                            enhanced_message_parts.append(f"❌ **Error processing file: {Path(file_path).name}**")
        
        # Create the final message content structure
        if has_images:
            # For messages with images, create content array with text and images
            text_content = "\n\n".join(enhanced_message_parts)
            final_content = [
                {
                    "type": "text",
                    "text": text_content
                }
            ]
            final_content.extend(image_contents)
            logger.info(f"Created message with {len(image_contents)} image(s)")
        else:
            # For text-only messages, use simple string format
            final_content = "\n\n".join(enhanced_message_parts)
        
        # Add current message
        messages.append({
            "role": "user", 
            "content": final_content
        })
        
        logger.info(f"Making API call with model: {CONFIG['model']}")
        logger.info(f"Message content type: {'array' if isinstance(final_content, list) else 'string'}")
        logger.info(f"Has images: {has_images}")
        
        # API call with vision support
        stream = anthropic_client.messages.create(
            model=CONFIG["model"],
            max_tokens=CONFIG["max_tokens"],
            temperature=CONFIG["temperature"],
            system=complete_system,
            messages=messages,
            stream=True
        )
        
        full_reply = ""
        for chunk in stream:
            if chunk.type == "content_block_delta":
                text = chunk.delta.text
                full_reply += text
                yield {"role": "assistant", "content": full_reply}
                
    except Exception as e:
        logger.error(f"Chat error: {e}")
        logger.error(f"Error details: {type(e).__name__}: {str(e)}")
        # Try to extract more specific error information
        if hasattr(e, 'response'):
            logger.error(f"API Response: {e.response}")
        error_msg = f"I apologize, but I'm experiencing technical difficulties: {str(e)}. Please try again."
        yield {"role": "assistant", "content": error_msg}

# Enhanced room selection with validation
def select_room(room_name, room_displays):
    if room_name not in ROOM_CONTEXTS:
        logger.warning(f"Invalid room selected: {room_name}")
        return gr.update(), gr.update(), gr.update(), "", room_displays
    
    room_intro = f"You are entering the {room_name}. {ROOM_CONTEXTS[room_name][:100]}..."
    room_chat_history = room_displays.get(room_name, [])
    
    return (
        gr.update(value=room_intro, visible=True),
        gr.update(value=room_chat_history, visible=True),
        gr.update(visible=True),
        room_name,
        room_displays
    )

# UPDATED: Enhanced user interaction with better image handling
def user_interaction(message, global_history, current_room, room_displays, uploaded_files):
    if not message.strip():
        return room_displays.get(current_room, []), global_history, room_displays, "", None
    
    if not current_room:
        error_display = [("Please select a room first.", "")]
        return error_display, global_history, room_displays, "", None
    
    try:
        # Convert global history for Claude (keep the existing format)
        claude_history = [{"role": turn["role"], "content": turn["content"]} for turn in global_history]
        
        # Update displays
        current_room_display = room_displays.get(current_room, []).copy()
        
        # Create enhanced message with file content for internal processing
        enhanced_message_parts = [message]
        display_message = message
        
        if uploaded_files:
            file_names = []
            text_file_contents = []
            has_images = False
            
            for file_path in uploaded_files:
                if file_path:
                    file_extension = Path(file_path).suffix.lower()
                    file_name = Path(file_path).name
                    file_names.append(file_name)
                    
                    if file_extension in ['.jpg', '.jpeg', '.png', '.gif', '.webp']:
                        has_images = True
                    else:
                        # Process non-image files as text
                        file_content, _ = process_uploaded_file(file_path)
                        if file_content:
                            text_file_contents.append(file_content)
            
            if file_names:
                # For display: show file names and indicate if images are present
                display_message += f"\n\n📎 **Attached files:** {', '.join(file_names)}"
                if has_images:
                    display_message += "\n🖼️ *Images will be analyzed by Falkor*"
                
                # For processing: add text file content (images handled separately in stream_chat)
                if text_file_contents:
                    enhanced_message_parts.extend(text_file_contents)
        
        # Add user message to room display (using tuple format for Gradio)
        current_room_display.append((display_message, ""))
        
        updated_room_displays = room_displays.copy()
        updated_room_displays[current_room] = current_room_display
        
        # Store enhanced message for text content, images handled in stream_chat
        enhanced_message = "\n\n".join(enhanced_message_parts) if len(enhanced_message_parts) > 1 else message
        updated_global_history = global_history + [{"role": "user", "content": enhanced_message}]
        
        # Show user message immediately
        yield current_room_display, updated_global_history, updated_room_displays, "", None
        
        # Stream response with both text and image content
        for chunk in stream_chat(message, claude_history, current_room, uploaded_files):
            full_reply = chunk["content"]
            # Update room display with response (using tuple format)
            current_room_display_with_reply = current_room_display[:-1] + [(display_message, full_reply)]
            updated_room_displays[current_room] = current_room_display_with_reply
            yield current_room_display_with_reply, updated_global_history, updated_room_displays, "", None
        
        # Final update with complete response
        final_global_history = updated_global_history + [{"role": "assistant", "content": full_reply}]
        yield current_room_display_with_reply, final_global_history, updated_room_displays, "", None
        
    except Exception as e:
        logger.error(f"User interaction error: {e}")
        error_msg = "I encountered an error processing your message. Please try again."
        # Use tuple format for error display
        error_display = current_room_display[:-1] + [(display_message, error_msg)]
        yield error_display, global_history, room_displays, "", None

# Initialize room displays
def initialize_room_displays():
    return {room: [] for room in ROOM_CONTEXTS.keys()}

# UI 
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.HTML("""
    <style>
        # UPDATED: Enhanced user interaction with better image handling
        #chat-window {
             height: calc(100vh - 250px); /* Adjusted back since file upload is now compact */
             overflow-y: auto;
             border: 1px solid #374151;
             border-radius: 10px;
             padding: 10px;
             background-color: #1a1d23;
        }   
        
        #input-row {
            position: fixed;
            bottom: 0;
            left: 0;
            right: 0;
            background-color: #0f1419;
            padding: 15px 20px;
            border-top: 1px solid #374151;
            z-index: 10;
        }

        .file-upload-container {
            position: relative;
            display: inline-flex;
            align-items: center;
            justify-content: center;
            width: 40px;
            height: 40px;
            margin-right: 10px;
        }

        .file-upload-button {
            position: absolute;
            top: -9px;
            left: -8px;
            width: 40px;
            height: 40px;
            background-color: #374151;
            border: 1px solid #4b5563;
            border-radius: 50%;
            cursor: pointer;
            transition: all 0.3s ease;
            display: flex;
            align-items: center;
            justify-content: center;
            pointer-events: none;
            z-index: 1;
        }

        .file-upload-button:hover,
        .file-upload-container:hover .file-upload-button {
            background-color: #4b5563;
            border-color: #d4af37;
            transform: scale(1.05);
        }

        .file-upload-button svg {
            width: 20px;
            height: 20px;
            color: #ffffff;
        }

        .file-upload-container:hover .file-upload-button svg {
            color: #d4af37;
        }

        /* Style the actual Gradio file upload to overlay the button */
        .compact-file-upload {
            position: absolute !important;
            top: 0 !important;
            left: 0 !important;
            width: 40px !important;
            height: 40px !important;
            z-index: 2 !important;
            opacity: 0 !important;
            cursor: pointer !important;
            overflow: hidden !important;
        }

        .compact-file-upload > div {
            width: 40px !important;
            height: 40px !important;
            border-radius: 50% !important;
            border: none !important;
            background: transparent !important;
            overflow: hidden !important;
        }

        .compact-file-upload input[type="file"] {
            width: 40px !important;
            height: 40px !important;
            cursor: pointer !important;
        }

        /* Aggressive scrollbar hiding for file upload area */
        .file-upload-container,
        .file-upload-container *,
        .file-upload-container div,
        .file-upload-container .gradio-group,
        .file-upload-container .gr-group,
        .file-upload-container .gr-form,
        .file-upload-container .gr-box,
        .file-upload-container .gr-file,
        .compact-file-upload,
        .compact-file-upload *,
        .compact-file-upload div {
            overflow: hidden !important;
            scrollbar-width: none !important;
            -ms-overflow-style: none !important;
            max-width: 40px !important;
            max-height: 40px !important;
        }

        .file-upload-container::-webkit-scrollbar,
        .file-upload-container *::-webkit-scrollbar,
        .file-upload-container div::-webkit-scrollbar,
        .file-upload-container .gradio-group::-webkit-scrollbar,
        .file-upload-container .gr-group::-webkit-scrollbar,
        .file-upload-container .gr-form::-webkit-scrollbar,
        .file-upload-container .gr-box::-webkit-scrollbar,
        .file-upload-container .gr-file::-webkit-scrollbar,
        .compact-file-upload::-webkit-scrollbar,
        .compact-file-upload *::-webkit-scrollbar,
        .compact-file-upload div::-webkit-scrollbar {
            display: none !important;
            width: 0 !important;
            height: 0 !important;
        }

        /* Force exact dimensions and hide any content overflow */
        .file-upload-container .gr-group > div,
        .file-upload-container .gradio-group > div {
            width: 40px !important;
            height: 40px !important;
            max-width: 40px !important;
            max-height: 40px !important;
            overflow: hidden !important;
            border: none !important;
            padding: 0 !important;
            margin: 0 !important;
        }

        .input-with-upload {
            display: flex;
            align-items: flex-end;
            gap: 10px;
        }

        /* File indicator when files are selected */
        .files-selected-indicator {
            position: absolute;
            top: -8px;
            right: -8px;
            background-color: #d4af37;
            color: #000;
            border-radius: 50%;
            width: 18px;
            height: 18px;
            font-size: 11px;
            font-weight: bold;
            display: flex;
            align-items: center;
            justify-content: center;
            z-index: 2;
        }

        @import url('https://fonts.googleapis.com/css2?family=Crimson+Text:ital,wght@0,400;0,600;1,400&display=swap');
         
        body {
            margin: 0;
            padding: 0;
            background-color: #0f1419;
            color: #ffffff;
            font-family: 'Crimson Text', 'Times New Roman', serif !important;
        }
        .gradio-container {
            background-color: #0f1419 !important;
        }
        /* Apply consistent font to all text elements */
        *, .gr-markdown, .gr-markdown p, .gr-markdown h1, .gr-markdown h2, .gr-markdown h3,
        .gr-textbox, .gr-button, .gr-tab, .gr-chatbot, p, h1, h2, h3, div {
            font-family: 'Crimson Text', 'Times New Roman', serif !important;
        }
        /* Gentle dark mode styling */
        .gr-form, .gr-box {
            background-color: #1a1d23 !important;
            border: 1px solid #374151 !important;
        }
        .gr-button {
            background-color: #374151 !important;
            color: #ffffff !important;
            border: 1px solid #4b5563 !important;
            font-family: 'Crimson Text', 'Times New Roman', serif !important;
        }
        .gr-button:hover {
            background-color: #4b5563 !important;
        }
        .gr-textbox {
            background-color: #1a1d23 !important;
            color: #ffffff !important;
            border: 1px solid #374151 !important;
            font-family: 'Crimson Text', 'Times New Roman', serif !important;
        }
        .gr-file {
            background-color: #1a1d23 !important;
            border: 1px solid #374151 !important;
        }
        .pulsing-logo {
            display: flex !important;
            justify-content: center !important;
            align-items: center !important;
            padding: 20px !important;
            margin: 20px 0 !important;
            overflow: visible !important;
            position: relative !important;
        }
            
        /* Aggressively target all Gradio image-related containers */
        .pulsing-logo *,
        .pulsing-logo .gr-image,
        .pulsing-logo [data-testid="image"],
        .pulsing-logo .gr-image > div,
        .pulsing-logo .gr-image > div > div,
        .pulsing-logo div[data-testid="image"] > div,
        .pulsing-logo div[data-testid="image"] > div > div {
            overflow: visible !important;
            border: none !important;
            background: transparent !important;
            box-shadow: none !important;
            clip-path: none !important;
            -webkit-clip-path: none !important;
        }
            
        .pulsing-logo img {
            animation: pulse-glow 3s ease-in-out infinite;
            border-radius: 50% !important;
            position: relative !important;
            z-index: 1 !important;
        }
            
        @keyframes pulse-glow {
            0%, 100% {
                filter: drop-shadow(0 0 8px rgba(255, 255, 255, 0.4));
                transform: scale(1);
            }
            50% {
                filter: drop-shadow(0 0 25px rgba(255, 255, 255, 0.7)) drop-shadow(0 0 40px rgba(255, 255, 255, 0.5));
                transform: scale(1.02);
            }
        }
        .temenos-title {
            font-family: 'Cinzel', serif;
            font-size: 48px;
            color: #e8e8f0;
            text-align: center;
            margin: 30px auto 10px auto;
            letter-spacing: 6px;
            text-shadow: 0 0 10px rgba(200, 200, 255, 0.4);
            animation: fadeIn 4s ease;
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(-10px); }
            to { opacity: 1; transform: translateY(0); }
        }
                   
        .welcome-box {
            background: rgba(30, 30, 40, 0.7);
            border: 1px solid rgba(200, 200, 255, 0.1);
            padding: 25px;
            border-radius: 12px;
            max-width: 800px;
            margin: 40px auto;
            box-shadow: 0 0 30px rgba(100, 100, 150, 0.2);
            font-size: 1.1em;
            line-height: 1.6;
            backdrop-filter: blur(8px);
            color: #ddd;
            text-align: center;
        }
    </style>
    """)

    welcome_page = gr.Group(visible=True) #laat simeplweg zien welke page als eerst vertoont wordt
    main_page = gr.Group(visible=False)
    
    # Falkor introduction modal
    falkor_modal = gr.HTML("""
    <div id="falkor-modal" style="display: none; position: fixed; z-index: 1000; left: 0; top: 0; width: 100%; height: 100%; background-color: rgba(0,0,0,0.8);">
        <div style="background-color: #1a1a1a; margin: 10% auto; padding: 30px; border: 2px solid #444; border-radius: 15px; width: 80%; max-width: 500px; text-align: center; color: #ffffff; font-family: 'Merriweather', serif;">
            <h2 style="color: #d4af37; margin-bottom: 20px;">🐉 Meet Falkor</h2>
            <p style="font-size: 16px; line-height: 1.6; margin-bottom: 20px;">
                Greetings, fellow traveler. I am Falkor, your guide through the depths of the psyche. 
                Together, we shall explore the symbolic realm of your unconscious, confronting shadows, 
                interpreting dreams, and walking the path toward individuation.
            </p>
            <p style="font-style: italic; margin-bottom: 25px; color: #ccc;">
                "Who looks outside, dreams; who looks inside, awakens." - C.G. Jung
            </p>
            <button onclick="document.getElementById('falkor-modal').style.display='none'" 
                    style="background-color: #d4af37; color: #000; border: none; padding: 10px 20px; border-radius: 5px; cursor: pointer; font-size: 16px; font-weight: bold;">
                Begin the Journey
            </button>
        </div>
    </div>
    
    <script>
        function showFalkorModal() {
            document.getElementById('falkor-modal').style.display = 'block';
        }
    </script>
    """, visible=True)

    # Welcome screen
    with welcome_page:
        with gr.Row():
            with gr.Column(elem_classes=["pulsing-logo"]):
                gr.Image(value=logo_path, show_label=False, container=False, interactive=False, show_download_button=False, show_fullscreen_button=False, width=200, height=200)
        
        gr.HTML("""
        <div class="temenos-title">TEMENOS</div>
        <div class="welcome-box">
          <p>
            Welcome to <strong>TEMENOS</strong> — your sacred inner space devoted to the process of individuation.  
            Guided by <strong>Falkor</strong>, your companion and Jungian guide,  
            you are invited to journey inward: to confront your shadows, decipher the language of your unconscious, and walk the path towards individuation.
          </p>
        </div>
        """)

        proceed_btn = gr.Button(" Enter the Temenos ")
        audio = gr.Audio(value=audio_path, autoplay=True, visible=False)

    # Main experience
    with main_page:
        with gr.Row():
            gr.Image(value=logo_path, show_label=False, container=False, interactive=False, show_download_button=False, show_fullscreen_button=False, width=120, height=120)

        gr.HTML("""
        <div style="text-align: center; margin-bottom: 20px; font-family: 'Crimson Text', 'Times New Roman', serif;">
            <h1 style="font-family: 'Crimson Text', 'Times New Roman', serif;">🐉 Temenos 🏰</h1>
            <p style="font-family: 'Crimson Text', 'Times New Roman', serif;">Choose a room to begin your journey of individuation.</p>
        </div>
        """)

        # Room selection buttons
        with gr.Row():
            shadow_btn = gr.Button("💀 Shadow Dungeon", variant="secondary", size="lg")
            dream_btn = gr.Button("🌙 Dream Chamber", variant="secondary", size="lg")
            imagination_btn = gr.Button("🧙‍♂ Alchemist's Workshop", variant="secondary", size="lg")
            mandala_btn = gr.Button("🌹 Mandala Garden", variant="secondary", size="lg")
            integration_btn = gr.Button("🏰 Castle Gates", variant="secondary", size="lg")

        # Current room display
        current_room_display = gr.Markdown("*Select a room to begin your journey with Falkor*", visible=True)
        
        # Chat interface
        chatbot = gr.Chatbot(elem_id="chat-window", show_label=False, visible=False)
        
        # Compact input area with integrated file upload
        with gr.Row(visible=False, elem_id="input-row") as chat_row:
            with gr.Column(scale=1):
                # Compact message input area with file upload button
                with gr.Row(elem_classes=["input-with-upload"]):
                    # File upload button (styled as + icon)
                    with gr.Column(scale=0, min_width=50):
                        with gr.Group(elem_classes=["file-upload-container"]):
                            # Visual button design
                            gr.HTML("""
                            <div class="file-upload-button">
                                <svg viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2">
                                    <line x1="12" y1="5" x2="12" y2="19"></line>
                                    <line x1="5" y1="12" x2="19" y2="12"></line>
                                </svg>
                                <div class="files-selected-indicator" id="file-count-indicator" style="display: none;">0</div>
                            </div>
                            """)
                            # Invisible but functional file upload overlaid on top
                            file_upload = gr.File(
                                label="",
                                file_count="multiple",
                                file_types=["text", "image", "audio", "video", ".pdf", ".json", ".py", ".js", ".html", ".css", ".md", ".docx", ".doc"],
                                container=False,
                                show_label=False,
                                elem_classes=["compact-file-upload"]
                            )
                    
                    # Message input
                    with gr.Column(scale=4):
                        msg_box = gr.Textbox(
                            placeholder="Speak to Falkor...", 
                            show_label=False, 
                            container=False,
                            lines=2
                        )
                    
                    # Send button
                    with gr.Column(scale=1, min_width=80):
                        send_btn = gr.Button("Send", variant="primary", size="lg")

        # Add JavaScript for file upload visual feedback
        gr.HTML("""
        <script>
        function setupFileUploadIndicator() {
            // Find the file input and indicator
            const fileInputs = document.querySelectorAll('.compact-file-upload input[type="file"]');
            const indicator = document.getElementById('file-count-indicator');
            
            fileInputs.forEach(function(fileInput) {
                if (!fileInput.hasChangeListener) {
                    fileInput.hasChangeListener = true;
                    fileInput.addEventListener('change', function() {
                        const fileCount = this.files.length;
                        const button = document.querySelector('.file-upload-button');
                        
                        if (fileCount > 0) {
                            if (indicator) {
                                indicator.textContent = fileCount;
                                indicator.style.display = 'flex';
                            }
                            if (button) {
                                button.style.borderColor = '#d4af37';
                                button.style.backgroundColor = '#4b5563';
                            }
                        } else {
                            if (indicator) {
                                indicator.style.display = 'none';
                            }
                            if (button) {
                                button.style.borderColor = '#4b5563';
                                button.style.backgroundColor = '#374151';
                            }
                        }
                    });
                }
            });
        }
        
        // Setup when the page loads and after content changes
        document.addEventListener('DOMContentLoaded', setupFileUploadIndicator);
        setTimeout(setupFileUploadIndicator, 1000);
        setInterval(setupFileUploadIndicator, 3000); // Periodic check
        </script>
        """)

        # Hidden states to track rooms and conversations
        current_room_state = gr.State("")
        global_chat_history = gr.State([])  # Stores ALL conversations across rooms for Claude's memory
        room_chat_displays = gr.State({     # Stores room-specific chat displays
            "Shadow Dungeon": [],
            "Dream Chamber": [],
            "Alchemist's Workshop": [],
            "Mandala Garden": [],
            "Castle Gates": []
        })

    # Room selection function - now manages room-specific displays
    room_display = {
        "Shadow Dungeon": "You are entering the Shadow Dungeon. Prepare to face repressed aspects of the self...",
        "Dream Chamber": "You sit in the Dream Chamber. Dreams rise like mist from the unconscious.",
        "Alchemist's Workshop": "You discover the Alchemist's Workshop. Prepare your transformative vessel.",
        "Mandala Garden": "You step into the Mandala Garden. Let unconscious content take symbolic form.",
        "Castle Gates": "Welcome to the castle gates. Prepare yourself for the quests which lie beyond."
    }   

    def select_room(room_name, room_displays):
        room_intro = room_display.get(room_name, "")
        # Get the chat history for this specific room
        room_chat_history = room_displays.get(room_name, [])
        
        return (
            gr.update(value=f"{room_intro}", visible=True),
            gr.update(value=room_chat_history, visible=True),  # Show room-specific chat
            gr.update(visible=True),
            room_name,
            room_displays
        )

    # Connect room selection buttons
    shadow_btn.click(
        fn=lambda room_displays: select_room("Shadow Dungeon", room_displays),
        inputs=[room_chat_displays],
        outputs=[current_room_display, chatbot, chat_row, current_room_state, room_chat_displays]
    )
    
    dream_btn.click(
        fn=lambda room_displays: select_room("Dream Chamber", room_displays),
        inputs=[room_chat_displays],
        outputs=[current_room_display, chatbot, chat_row, current_room_state, room_chat_displays]
    )
    
    imagination_btn.click(
        fn=lambda room_displays: select_room("Alchemist's Workshop", room_displays),
        inputs=[room_chat_displays],
        outputs=[current_room_display, chatbot, chat_row, current_room_state, room_chat_displays]
    )
    
    mandala_btn.click(
        fn=lambda room_displays: select_room("Mandala Garden", room_displays),
        inputs=[room_chat_displays],
        outputs=[current_room_display, chatbot, chat_row, current_room_state, room_chat_displays]
    )
    
    integration_btn.click(
        fn=lambda room_displays: select_room("Castle Gates", room_displays),
        inputs=[room_chat_displays],
        outputs=[current_room_display, chatbot, chat_row, current_room_state, room_chat_displays]
    )

    # Connect chat interactions with file support
    msg_box.submit(
        fn=user_interaction,
        inputs=[msg_box, global_chat_history, current_room_state, room_chat_displays, file_upload],
        outputs=[chatbot, global_chat_history, room_chat_displays, msg_box, file_upload]
    )
    
    send_btn.click(
        fn=user_interaction,
        inputs=[msg_box, global_chat_history, current_room_state, room_chat_displays, file_upload],
        outputs=[chatbot, global_chat_history, room_chat_displays, msg_box, file_upload]
    )

    # Button action to switch views + play audio + show Falkor modal
    def go_to_main():
        return (
            gr.update(visible=False), 
            gr.update(visible=True), 
            gr.update(visible=True),
            gr.HTML("""
            <div id="falkor-modal" style="display: block; position: fixed; z-index: 1000; left: 0; top: 0; width: 100%; height: 100%; background-color: rgba(0,0,0,0.8);">
                <div style="background-color: #1a1a1a; margin: 10% auto; padding: 30px; border: 2px solid #444; border-radius: 15px; width: 80%; max-width: 500px; text-align: center; color: #ffffff; font-family: 'Merriweather', serif;">
                    <h2 style="color: #d4af37; margin-bottom: 20px;">🐉 Meet Falkor</h2>
                    <p style="font-size: 16px; line-height: 1.6; margin-bottom: 20px;">
                        Greetings, fellow traveler. I am Falkor, your guide through the depths of the psyche. Together, we shall explore the symbolic realm of your unconscious, confront shadows, interpret dreams, and walk the path towards individuation.
                    </p>
                    <p style="font-style: italic; margin-bottom: 25px; color: #ccc;">
                        "Who looks outside, dreams; who looks inside, awakens." - C.G. Jung
                    </p>
                    <button onclick="this.parentElement.parentElement.style.display='none'" 
                            style="background-color: #d4af37; color: #000; border: none; padding: 10px 20px; border-radius: 5px; cursor: pointer; font-size: 16px; font-weight: bold;">
                        Begin the Journey
                    </button>
                </div>
            </div>
            """)
        )

    proceed_btn.click(fn=go_to_main, inputs=[], outputs=[welcome_page, main_page, audio, falkor_modal])

# Run the app
if __name__ == "__main__": 
    demo.launch(share=True)

INFO:__main__:Loaded RAG index with 18628 documents
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_13412\2948044167.py:918: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(elem_id="chat-window", show_label=False, visible=False)
INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7882


INFO:httpx:HTTP Request: GET http://127.0.0.1:7882/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7882/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"


* Running on public URL: https://70099d6b58f6759acb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


INFO:httpx:HTTP Request: HEAD https://70099d6b58f6759acb.gradio.live "HTTP/1.1 200 OK"


C:\Users\Gebruiker\AppData\Roaming\Python\Python313\site-packages\gradio\blocks.py:2010: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  state[block._id] = block.__class__(**kwargs)
